In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
import warnings
import locale
import os
from tkinter import messagebox
from datetime import datetime

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

hoy = datetime.now()
año = hoy.year
mes = hoy.month

mes_nombre = hoy.strftime('%b').upper()[:3] # ENE
mes_año = f'{mes_nombre}{str(año)[2:]}' # ENE24
fecha = f'{año}{str(mes).zfill(2)}' # 202401

reporte_condonacion = f'bases/condonaciones/Reporte de Condonación.csv'
base_final = f'bases/condonaciones/Reporte_Condonaciones_{mes_año}.xlsx'
base_final_path = os.path.abspath(base_final)

print('Reporte de Condonación:', reporte_condonacion)
print('Base Final:', base_final_path)

In [ ]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [ ]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar reporte de condonación?", icon='warning')

if result == 'yes':
    df_base = pd.read_csv(
        reporte_condonacion, 
        sep=';', 
        encoding='utf-8', 
        dtype={'Marca temporal': str, 'CC': str, 'N° del contrato de cliente / código UGA': str, 'Fecha de revisión': str, 'Fecha de ejecución': str})
    df_base.columns = clean_columns(df_base.columns)
    print('Base Condonaciones:', df_base.shape)

root.destroy()

In [ ]:
df_base

In [ ]:
df_base_test = df_base.copy()

cols_required = [
    'SELECCIONAR_LA_AGENCIA_DE_COBRANZA_/_UNIDAD_CORRESPONDIENTE', 
    'CC', 
    'N°_DEL_CONTRATO_DE_CLIENTE_/_CÓDIGO_UGA', 
    'MARCA_TEMPORAL', 
    'FECHA_DE_REVISIÓN', 
    '¿REVISIÓN_CONFORME?', 
    'FECHA_DE_EJECUCIÓN', 
    'EJECUCIÓN'
    ]

df_base_test = df_base_test[cols_required]
df_base_test.rename(columns={
    'MARCA_TEMPORAL': 'FECHA_INGRESO', 
    'SELECCIONAR_LA_AGENCIA_DE_COBRANZA_/_UNIDAD_CORRESPONDIENTE': 'AGENCIA', 
    'N°_DEL_CONTRATO_DE_CLIENTE_/_CÓDIGO_UGA': 'CONTRATO', 
    'FECHA_DE_EJECUCIÓN': 'FECHA_EJECUCION', 
    '¿REVISIÓN_CONFORME?': 'CONFORME', 
    'FECHA_DE_REVISIÓN': 'FECHA_REVISION', 
    'EJECUCIÓN': 'EJECUCION'
    }, inplace=True)

df_base_test['AGENCIA'] = df_base_test['AGENCIA'].str.upper()
df_base_test['EJECUCION'] = df_base_test['EJECUCION'].str.upper()
df_base_test['CONFORME'] = df_base_test['CONFORME'].str.upper()
df_base_test['CC'] = df_base_test['CC'].str.zfill(8)
df_base_test['CONTRATO'] = df_base_test['CONTRATO'].str.zfill(18)

df_base_test['FECHA_INGRESO'] = pd.to_datetime(df_base_test['FECHA_INGRESO'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
df_base_test['FECHA_REVISION'] = pd.to_datetime(df_base_test['FECHA_REVISION'], format='%d/%m/%Y', errors='coerce').dt.date
df_base_test['FECHA_EJECUCION'] = pd.to_datetime(df_base_test['FECHA_EJECUCION'], format='%d/%m/%Y', errors='coerce').dt.date

df_base_test = df_base_test[df_base_test['FECHA_INGRESO'].dt.year >= 2024]

df_base_test = df_base_test[df_base_test['AGENCIA'].isin(['RJ ABOGADOS', 'CLASA', 'MORNESE'])]
df_base_test['AGENCIA'] = df_base_test['AGENCIA'].replace('RJ ABOGADOS', 'ASESCOM RJ')

df_base_test = df_base_test[df_base_test['FECHA_REVISION'].notnull()]
df_base_test = df_base_test[~((df_base_test['EJECUCION'] == 'SI') & (df_base_test['FECHA_EJECUCION'].isnull()))]
df_base_test = df_base_test[~((df_base_test['EJECUCION'].isnull()) & (df_base_test['FECHA_EJECUCION'].notnull()))]
df_base_test = df_base_test[~((df_base_test['EJECUCION'].notnull()) & (df_base_test['FECHA_EJECUCION'].isnull()))]
df_base_test = df_base_test[~((df_base_test['CONFORME'] == 'NO') & (df_base_test['EJECUCION'] == 'SI'))]
df_base_test = df_base_test[~((df_base_test['CONFORME'] == 'SI') & (df_base_test['EJECUCION'] == 'NO'))]
df_base_test = df_base_test[~((df_base_test['CONFORME'] == 'SI') & (df_base_test['EJECUCION'].isnull()))]
df_base_test['EJECUCION'].fillna('NO', inplace=True)

df_base_test.sort_values(by=['CONTRATO', 'FECHA_INGRESO'], ascending=True, inplace=True)
df_base_test.reset_index(drop=True, inplace=True)

df_base_test['FECHA_INGRESO'] = df_base_test['FECHA_INGRESO'].dt.strftime('%d-%m-%Y %H:%M:%S')
df_base_test['FECHA_INGRESO'] = pd.to_datetime(df_base_test['FECHA_INGRESO'].str.split(' ').str[0], format='%d-%m-%Y', errors='coerce').dt.date

print(df_base_test.shape)
df_base_test.head(5)

In [ ]:
df_base_test[['CONFORME', 'EJECUCION']].value_counts()

In [ ]:
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday

class Feriados(AbstractHolidayCalendar):
    rules = [
        # Días fijos
        Holiday('Año Nuevo', month=1, day=1),
        Holiday('Día del trabajo', month=5, day=1),
        Holiday('San Pedro y San Pablo', month=6, day=29),
        Holiday('Día de la independencia', month=7, day=28),
        Holiday('Día de la independencia', month=7, day=29),
        Holiday('Santa Rosa de Lima', month=8, day=30),
        Holiday('Combate de Angamos', month=10, day=8),
        Holiday('Día de todos los santos', month=11, day=1),
        Holiday('Inmaculada Concepción', month=12, day=8),
        Holiday('Batalla de Ayacucho', month=12, day=9),
        Holiday('Navidad', month=12, day=25),
        
        # Semana santa
        Holiday('Jueves Santo', month=4, day=6, year=2023),
        Holiday('Viernes Santo', month=4, day=7, year=2023),
        Holiday('Jueves Santo', month=3, day=28, year=2024),
        Holiday('Viernes Santo', month=3, day=29, year=2024),
        Holiday('Jueves Santo', month=4, day=18, year=2025),
        Holiday('Viernes Santo', month=4, day=19, year=2025),
        Holiday('Jueves Santo', month=4, day=2, year=2026),
        Holiday('Viernes Santo', month=4, day=3, year=2026),
        Holiday('Jueves Santo', month=3, day=25, year=2027),
        Holiday('Viernes Santo', month=3, day=26, year=2027),
        Holiday('Jueves Santo', month=4, day=13, year=2028),
        Holiday('Viernes Santo', month=4, day=14, year=2028),
        Holiday('Jueves Santo', month=3, day=29, year=2029),
        Holiday('Viernes Santo', month=3, day=30, year=2029),
        
        # Otros feriados
        Holiday('Día adicional', month=7, day=23, year=2023),
        Holiday('Día adicional', month=8, day=6, year=2023),
        Holiday('Día adicional', month=10, day=8, year=2023),
        Holiday('Día adicional', month=12, day=9, year=2023),
        Holiday('Día adicional', month=6, day=7, year=2024),
        Holiday('Día adicional', month=7, day=23, year=2024),
        Holiday('Día adicional', month=8, day=6, year=2024),
        Holiday('Día adicional', month=9, day=8, year=2024),
        Holiday('Día adicional', month=6, day=7, year=2025),
        Holiday('Día adicional', month=7, day=23, year=2025),
        Holiday('Día adicional', month=8, day=6, year=2025),
        Holiday('Día adicional', month=10, day=8, year=2025),
        Holiday('Día adicional', month=12, day=9, year=2025),
        Holiday('Día adicional', month=6, day=7, year=2026),
        Holiday('Día adicional', month=7, day=23, year=2026),
        Holiday('Día adicional', month=8, day=6, year=2026),
        Holiday('Día adicional', month=10, day=8, year=2026),
        Holiday('Día adicional', month=12, day=9, year=2026),
        Holiday('Día adicional', month=6, day=7, year=2027),
        Holiday('Día adicional', month=7, day=23, year=2027),
        Holiday('Día adicional', month=8, day=6, year=2027),
        Holiday('Día adicional', month=10, day=8, year=2027),
        Holiday('Día adicional', month=12, day=9, year=2027),
        Holiday('Día adicional', month=6, day=7, year=2028),
        Holiday('Día adicional', month=7, day=23, year=2028),
        Holiday('Día adicional', month=8, day=6, year=2028),
        Holiday('Día adicional', month=10, day=8, year=2028),
        Holiday('Día adicional', month=12, day=9, year=2028),
        Holiday('Día adicional', month=6, day=7, year=2029),
        Holiday('Día adicional', month=7, day=23, year=2029),
        Holiday('Día adicional', month=8, day=6, year=2029),
        Holiday('Día adicional', month=10, day=8, year=2029),
        Holiday('Día adicional', month=12, day=9, year=2029),
    ]

In [ ]:
def get_dias_habiles(fecha_primer_ingreso, fecha_ejecucion):
    feriados = Feriados().holidays(fecha_primer_ingreso, fecha_ejecucion).to_pydatetime().tolist()
    feriados = [d.date() for d in feriados]
    return np.busday_count(fecha_primer_ingreso, fecha_ejecucion, holidays=feriados)

def calcular_dias_habiles(df: pd.DataFrame) -> pd.DataFrame:
    resultados = []
    
    df_agrupado = df.groupby('CONTRATO')
    for contrato, grupo in df_agrupado:
        fecha_primer_ingreso = None
        fecha_revision = None
        fecha_ejecucion = None
        agencia = None
        reprocesos = 0
        for _, row in grupo.iterrows():
            if fecha_primer_ingreso is None:
                fecha_primer_ingreso = row['FECHA_INGRESO']
            if row['EJECUCION'] == 'SI':
                fecha_revision = row['FECHA_REVISION']
                fecha_ejecucion = row['FECHA_EJECUCION']
                agencia = row['AGENCIA']
                break
            reprocesos += 1
        if fecha_primer_ingreso and fecha_ejecucion:
            dias_habiles = get_dias_habiles(fecha_primer_ingreso, fecha_ejecucion)
            resultados.append({
                'CC': row['CC'],
                'CONTRATO': contrato,
                'AGENCIA': agencia,
                'FECHA_INGRESO': fecha_primer_ingreso,
                'FECHA_REVISION': fecha_revision,
                'FECHA_EJECUCION': fecha_ejecucion,
                'DIAS_HABILES': dias_habiles,
                'REPROCESO': reprocesos
            })
    
    return pd.DataFrame(resultados)

In [ ]:
df_base_test

In [ ]:
df_dias_habiles = calcular_dias_habiles(df_base_test)
df_dias_habiles.head(5)

In [ ]:
df_dias_habiles

In [ ]:
cols_order = ['CC', 'CONTRATO', 'AGENCIA', 'FECHA_INGRESO', 'FECHA_EJECUCION', 'DIAS_HABILES', 'REPROCESO']
df_dias_habiles = df_dias_habiles[cols_order]
df_dias_habiles.sort_values(by=['FECHA_INGRESO', 'FECHA_EJECUCION'], ascending=True, inplace=True)
df_dias_habiles.reset_index(drop=True, inplace=True)
df_dias_habiles.head(5)

In [ ]:
df_dias_habiles.to_excel(base_final_path, index=False)
os.startfile(base_final_path)